# Hello, FABRIC: Create your first FABRIC slice

## Configure the Environment

Set the environment variable that will be used by this notebook. If you are using the FABRIC, JupyterHub some of the environment will be automatically configured for you.  You will only need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  If you do not have a bastion username and private key, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

In [1]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token_beta.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth_0031379841'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/FABRIC/pruth-bastion-1'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import the FABRIC API

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

#### Create the FABRIC Proxies

The FABRIC API is used via proxy objects that manage connections to the control framework.  

#### (Optional) Query Available Resources

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
    available_resources.draw()
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

## Create the Experiment Slice

Load aggragate ads.

In [3]:
import glob
import json

aggregates = {}
aggregate_files = glob.glob("./aggregates/*.json")
for file_name in aggregate_files:
    print(f"Reading {file_name}")
    site_name = file_name.replace("./aggregates/","",1).split(".")[0]
    #print(f"{site_name}")
    with open(file_name, 'r') as infile:
        site_info = json.load(infile)
    #print(f"{site_json}")
    aggregates[site_name] = site_info
    
#for site,site_info in aggregates.items():
#    print(f"{site}")
#    for node_info in site_info['Nodes']:
#        print(f"{node_info['Name']}")
#        for component_info in node_info['Components']:
#            print(f"{component_info['Description']}")

Reading ./aggregate-ads/json/RENC.json
Reading ./aggregate-ads/json/TACC.json
Reading ./aggregate-ads/json/MAX.json
Reading ./aggregate-ads/json/LBNL.json
Reading ./aggregate-ads/json/NCSA.json
Reading ./aggregate-ads/json/UTAH.json
Reading ./aggregate-ads/json/UKY.json
Reading ./aggregate-ads/json/SALT.json
Reading ./aggregate-ads/json/MICH.json
Reading ./aggregate-ads/json/STAR.json
Reading ./aggregate-ads/json/DALL.json
Reading ./aggregate-ads/json/WASH.json


#### Configure the Experiment Parameters



In [4]:
slice_name_base = 'MySlice'
#test_sites = [ 'UKY', 'RENC', 'LBNL' ]
#test_sites = [ 'TACC', 'MAX', 'UTAH', 'NCSA', 'MICH', 'WASH', 'DALL', 'SALT']
test_sites = [ 'TACC' ]
node_name_base = 'node'
#image = 'default_centos_8'
image = 'default_ubuntu_20'
#image = 'default_rocky_8'
cores = 2
ram = 8
disk = 10

node_count=5



In [5]:
#tests = {}
#for site in sites:
#    workers = []
#    for worker_num in range(1,4):
#        workers.append(f"{site.lower()}-w{worker_num}.fabric-testbed.net")
#    tests[site] = workers

#print(f"{tests}")


for site in test_sites:
    print(f"{site}")
    site_info = aggregates[site]
    for node_info in site_info['Nodes']:
        print(f"{node_info['Name']}")
        #for component_info in node_info['Components']:
        #    print(f"{component_info['Description']}")


TACC
tacc-w1.fabric-testbed.net
tacc-w2.fabric-testbed.net
tacc-w3.fabric-testbed.net
tacc-w4.fabric-testbed.net
tacc-w5.fabric-testbed.net


### Create Slice

<img src="./figs/SingleNode.png" width="20%"><br>

Create a single node with basic compute capabilities. The submit function will block until the node is ready and will display a progress bar.


In [6]:
for site in test_sites:
    print(f"{site}")
    site_info = aggregates[site]
    for node_info in site_info['Nodes']:
        worker = node_info['Name']
        worker_short_name = worker.split(".")[0]
        print(f"{worker}")
        try:
            slice_name=f"{slice_name_base}_{worker_short_name}"
            print(f"Creating slice {slice_name} on {worker}")
            #Create Slice
            slice = fablib.new_slice(slice_name)

            for node_num in range(node_count):
                node_name = f"{node_name_base}{node_num}"
                # Add node
                node = slice.add_node(name=node_name, site=site)
                node.set_capacities(cores=cores, ram=ram, disk=disk)
                node.set_image(image)
                node.set_host(worker)

            #Submit Slice Request
            slice.submit(wait_progress=False)
        except Exception as e:
            print(f"Slice Failed: {e}")
            #traceback.print_exc()
       

TACC
tacc-w1.fabric-testbed.net
Creating slice MySlice_tacc-w1 on tacc-w1.fabric-testbed.net
tacc-w2.fabric-testbed.net
Creating slice MySlice_tacc-w2 on tacc-w2.fabric-testbed.net
tacc-w3.fabric-testbed.net
Creating slice MySlice_tacc-w3 on tacc-w3.fabric-testbed.net
tacc-w4.fabric-testbed.net
Creating slice MySlice_tacc-w4 on tacc-w4.fabric-testbed.net
tacc-w5.fabric-testbed.net
Creating slice MySlice_tacc-w5 on tacc-w5.fabric-testbed.net


In [8]:
for site in test_sites:
    print(f"{site}")
    site_info = aggregates[site]
    for node_info in site_info['Nodes']:
        worker = node_info['Name']
        worker_short_name = worker.split(".")[0]
        print(f"{worker}")
        try:
            slice_name=f"{slice_name_base}_{worker_short_name}"
            print(f"Waiting for slice {slice_name} at {worker}")

            #Get Slice
            slice = fablib.get_slice(slice_name)
            
            #Wait for ssh to be active
            slice.wait_ssh(progress=True)
            
            #Run post boo config
            slice.post_boot_config(verbose=False)
        except Exception as e:
            print(f"Slice Failed: {e}")
            #traceback.print_exc()

TACC
tacc-w1.fabric-testbed.net
Waiting for slice MySlice_tacc-w1 at tacc-w1.fabric-testbed.net
Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful
post_boot_config
build_interface_map
Node data node0
{}
Node data node1
{}
Node data node2
{}
Node data node3
{}
Node data node4
{}
tacc-w2.fabric-testbed.net
Waiting for slice MySlice_tacc-w2 at tacc-w2.fabric-testbed.net
Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful
post_boot_config
build_interface_map
Node data node0
{}
Node data node1
{}
Node data node2
{}
Node data node3
{}
Node data node4
{}
tacc-w3.fabric-testbed.net
Waiting for slice MySlice_tacc-w3 at tacc-w3.fabric-testbed.net
Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful
post_boot_config
build_interface_map
Node data node0
{}
Node data node1
{}
Node data node2
{}
Node data node3
{}
Node data node4
{}
tacc-w4.fabric-testbed.net
Waiting for slice MySlice_tacc-w4 at tacc-w4.fa

### Print the Node's Attributes

Each node in the slice has a set of get functions that return the node's attributes.

In [ ]:
for site in test_sites:
    print(f"{site}")
    site_info = aggregates[site]
    for node_info in site_info['Nodes']:
        worker = node_info['Name']
        worker_short_name = worker.split(".")[0]
        print(f"{worker}")
        try:
            slice_name=f"{slice_name_base}_{worker_short_name}"
            print(f"Get attributes for {slice_name} at {site}")    
            slice = fablib.get_slice(slice_name)
            for node in slice.get_nodes():
                print("Node:")
                print(f"   Name              : {node.get_name()}")
                print(f"   Cores             : {node.get_cores()}")
                print(f"   RAM               : {node.get_ram()}")
                print(f"   Disk              : {node.get_disk()}")
                print(f"   Image             : {node.get_image()}")
                print(f"   Image Type        : {node.get_image_type()}")
                print(f"   Host              : {node.get_host()}")
                print(f"   Site              : {node.get_site()}")
                print(f"   Management IP     : {node.get_management_ip()}")
                print(f"   Reservation ID    : {node.get_reservation_id()}")
                print(f"   Reservation State : {node.get_reservation_state()}")
                print(f"   Components        : {node.get_components()}")
                print(f"   Interfaces        : {node.get_interfaces()}")
                print(f"   SSH Command       : {node.get_ssh_command()}")
                
                for component in node.get_components():
                    print(f"      Name             : {component.get_name()}")
                    print(f"      Details          : {component.get_details()}")
                    print(f"      Disk (G)         : {component.get_disk()}")
                    print(f"      Units            : {component.get_unit()}")
                    print(f"      PCI Address      : {component.get_pci_addr()}")
                    print(f"      Model            : {component.get_model()}")
                    print(f"      Type             : {component.get_type()}") 
                print(f"   Interfaces        :  ")
                for interface in node.get_interfaces():
                    print(f"       Name                : {interface.get_name()}")
                    print(f"           Bandwidth           : {interface.get_bandwidth()}")
                    print(f"           VLAN                : {interface.get_vlan()}")       
                    print(f"           OS Interface        : {interface.get_os_interface()}")       
            
                print()    
        except Exception as e:
            print(f"Fail: {e}")

## Accessing the VM


#### SSH with fablib

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [ ]:
command= 'echo Hello, FABRIC from node `hostname -s`'

In [ ]:
for site,workers in tests.items():
    for worker in workers:
        try:
            slice_name=f"{slice_name_base}_{worker}"
            print(f"Executing command for {slice_name} at {site}")    
            slice = fablib.get_slice(slice_name)
            for node in slice.get_nodes():
                stdout, stderr = node.execute(command)
                print(stdout)
        except Exception as e:
            print(f"Fail: {e}")

## Delete Slice

Please delete your slicd when you are done with your experiment.

In [ ]:
for site,workers in tests.items():
    for worker in workers:
        try:
            slice_name=f"{slice_name_base}_{worker}"

            print(f"Deleting for {slice_name} at {site}")    
            slice = fablib.get_slice(slice_name)
            slice.delete()
        except Exception as e:
            print(f"Fail: {e}")